In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(
    n_samples = 1000,
    n_features = 10,
    n_informative = 2,
    n_redundant = 8,
    weights = [0.9, 0.1],
    flip_y = 0,
    random_state = 42
)
np.unique(y, return_counts = True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 42)

### Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C = 1, solver = 'liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators = 30, max_depth = 3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state = 42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts = True)

(array([0, 1]), array([619, 619], dtype=int64))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       270
           1       0.86      0.83      0.85        30

    accuracy                           0.97       300
   macro avg       0.92      0.91      0.92       300
weighted avg       0.97      0.97      0.97       300



<div style="text-align: center;">
    <h1>Track Experiments using MLFlow</h1>
</div>

In [9]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ), 
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier with SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict = True)
    reports.append(report)

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

In [14]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Anomaly-Detection")

# Define the signature manually
input_schema = Schema([ColSpec("double", f"feature_{i+1}") for i in range(X_test.shape[1])])
output_schema = Schema([ColSpec("integer")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(
                model, 
                "model",
                signature=signature,  # Use the manually defined signature
                pip_requirements=["xgboost==2.1.3", "scikit-learn", "numpy", "pandas"]
            )
        else:
            mlflow.sklearn.log_model(
                model, 
                "model", 
                signature=signature  # Use the manually defined signature
            )  


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/775241755285561654/runs/6b66d8d13934447f800933656da0e6bc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/775241755285561654
🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/775241755285561654/runs/4bff46be8e6c403882de8b2ac359d901
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/775241755285561654
🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/775241755285561654/runs/9f30fe4d919c4e87913dc7991245a72f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/775241755285561654
🏃 View run XGBClassifier with SMOTE at: http://127.0.0.1:5000/#/experiments/775241755285561654/runs/b5614aee616e42b88c529e1fb27958f4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/775241755285561654


### Register the model

In [15]:
model_name = "XGB-Smote"
run_id = input("Enter Run ID: ")
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(
    model_uri, model_name
)

Enter Run ID:  b5614aee616e42b88c529e1fb27958f4


Successfully registered model 'XGB-Smote'.
2025/01/31 20:03:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


### Load the model

In [16]:
#prediction using dev model
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Production Model

In [18]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri = dev_model_uri, dst_name = prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1738339981964, current_stage='None', description='', last_updated_timestamp=1738339981964, name='anomaly-detection-prod', run_id='b5614aee616e42b88c529e1fb27958f4', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [20]:
#prediction using production model
model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])